In [284]:
import sqlite3, os, json, sqlparse, re
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from nltk.stem import WordNetLemmatizer
from sql_metadata import Parser

In [171]:
folder_path = "src/spider/database"
select_db = ['musical',
             'farm', 
             'hospital_1', 
             'tvshow', 
             'cinema', 
             'restaurants', 
             'company_employee', 
             'company_1', 
             'company_offic', 
             'singer', 
             'coffee_shop']

db = []

if os.path.exists(folder_path) and os.path.isdir(folder_path):
    files = os.listdir(folder_path)
    for file in files:
        # if file in select_db:
        db_path = os.path.join(folder_path, file)
        sqlite_db = [os.path.join(db_path, sql) for sql in os.listdir(db_path) if ".sqlite" in sql]
        db.append(*sqlite_db)

In [173]:
def get_schema(sqlite_db):
    connection = sqlite3.connect(sqlite_db)
    cursor = connection.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    for table in tables:
        table_name = table[0]
        print(f"Table: {table_name}")

        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()

        for column in columns:
            column_name = column[1]
            print(f"  Column: {column_name}")

        print()
    
    cursor.close()
    connection.close()

In [179]:
for table in db:
    # if table in exists_table : continue
    get_schema(table)
    # exists_table.append(table)
    print('---------------------------------')

Table: Web_client_accelerator
  Column: id
  Column: name
  Column: Operating_system
  Column: Client
  Column: Connection

Table: browser
  Column: id
  Column: name
  Column: market_share

Table: accelerator_compatible_browser
  Column: accelerator_id
  Column: browser_id
  Column: compatible_since_year

---------------------------------
Table: musical
  Column: Musical_ID
  Column: Name
  Column: Year
  Column: Award
  Column: Category
  Column: Nominee
  Column: Result

Table: actor
  Column: Actor_ID
  Column: Name
  Column: Musical_ID
  Column: Character
  Column: Duration
  Column: age

---------------------------------
Table: city
  Column: City_ID
  Column: Official_Name
  Column: Status
  Column: Area_km_2
  Column: Population
  Column: Census_Ranking

Table: farm
  Column: Farm_ID
  Column: Year
  Column: Total_Horses
  Column: Working_Horses
  Column: Total_Cattle
  Column: Oxen
  Column: Bulls
  Column: Cows
  Column: Pigs
  Column: Sheep_and_Goats

Table: farm_competition

In [175]:
src_folder = "src"
schema_description_file = "mockup_schema_description.json"
with open(os.path.join(src_folder, schema_description_file)) as f:
    dbs = json.load(f)
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
lemmanizer = WordNetLemmatizer()

In [56]:
# description_emb = []

# for db in dbs:
#     schema_emb = {}
#     table_name = db['table']
#     table_description = db['description']
#     schema_emb[table_name] = model.encode(table_description).tolist()
#     columns = list(db['columns'].keys())
#     for col in columns:
#         column_description = db['columns'][col]
#         schema_emb[col] = model.encode(column_description).tolist()
#     description_emb.append(schema_emb)

# schema_vector_file = "mockup_schema_description_vector.json"
# with open(os.path.join(src_folder, schema_vector_file), "w") as f:
#     json.dump(description_emb,f)

In [180]:
schema_vector_file = "mockup_schema_description_vector.json"
with open(os.path.join(src_folder, schema_vector_file)) as f:
    schema_vector = json.load(f)

In [376]:
# string mathing , max_score = weight of string match

def column_from_question(question,used_table_col = {}, default_score=0.6):
    # question_tokens = [token.lower() for token in tokenizer.tokenize(question)]
    question_tokens = [lemmanizer.lemmatize(token.lower()) for token in question.split()]
    # print(question_tokens)
    for table in schema_vector:
        max_score = default_score
        for token in question_tokens:
            cols = [ key.lower() for key in table.keys()]
            table_name = cols.pop(0)
            if token == table_name: 
                max_score = 1.0
                # plus the score of columns in exact match with table name
                if used_table_col.get(token) is not None: 
                    for key in used_table_col[token]: 
                        used_table_col[token].update({key: used_table_col[token][key] + 0.1})
            # exact match table and column
            if token in cols: 
                used_table_col.setdefault(table_name, {}).update({token : max_score})

            

    return used_table_col

In [365]:
# filter_table : filter by table before 
def filter_tables_by_description(question, column_threshold = 0.4, table_threshold = 0.2, filter_tables = True):
    question_emb = model.encode(question)
    used_schema = {}
    for i in range(len(schema_vector)):
        table_name = list(schema_vector[i].keys())[0]

        table_description_vector = schema_vector[i][table_name]
        if filter_tables and util.cos_sim(table_description_vector, question_emb) < table_threshold: continue
        
        used_col = {}
        for col, vec in schema_vector[i].items():
            if col == table_name: continue
            score = round(float(util.cos_sim(vec, question_emb)),2)
            if score > column_threshold:
                # column_description = [dbs[i]['columns'][col] for i in range(len(dbs)) if dbs[i]['table'] == table_name][0]
                # print(f"{table_name} - {col} : {score}\nDescription : {column_description}\n")
                used_col.update({col: score})
        if len(used_col) > 0: used_schema[table_name] = used_col
    return used_schema

In [366]:
question = "Who direct the mickey mouse"

In [384]:
selected_table_column = filter_tables_by_description(question, column_threshold = 0.3, filter_tables = False)
selected_table_column

{'Cartoon': {'id': 0.31,
  'Title': 0.35,
  'Directed_by': 0.54,
  'Written_by': 0.44,
  'Original_air_date': 0.32,
  'Production_code': 0.39},
 'film': {'Directed_by': 0.36}}

In [385]:
selected_table_column = column_from_question(question, used_table_col=selected_table_column)
selected_table_column

['who', 'direct', 'the', 'mickey', 'mouse']


{'Cartoon': {'id': 0.31,
  'Title': 0.35,
  'Directed_by': 0.54,
  'Written_by': 0.44,
  'Original_air_date': 0.32,
  'Production_code': 0.39},
 'film': {'Directed_by': 0.36}}

In [366]:
full_sql = ""
for table in selected_table_column:
    sql = f"CREATE TABLE {table} ("
    for column in selected_table_column.get(table).keys():
        sql += f'"{column}" datatype, '

    sql = sql[:-2] + " )\n\n"
    full_sql += sql
print(full_sql)

CREATE TABLE people ("Nationality" datatype )

CREATE TABLE singer ("Singer_ID" datatype, "Name" datatype, "Birth_Year" datatype, "Net_Worth_Millions" datatype, "Citizenship" datatype )

CREATE TABLE song ("Singer_ID" datatype )




In [27]:
spider_sql = []
df_data = {
    'Question' : [],
    'Table' : [],
    'SQL' : []
}

with open("src/NSText2SQL/train.jsonl") as f:
    for line in f:
        data = json.loads(line)
        if data['source'] == 'spider': 
            spider_sql.append(data)
            df_data['Question'].append(data['instruction'].split('--')[-1].strip())
            df_data['Table'].append(data['instruction'].split('--')[0].strip())
            df_data['SQL'].append(data['output'])

df = pd.DataFrame(df_data)
df.to_csv('src/NSText2SQL/train_spider.csv', index=False)
df.head()

,Question,Table,SQL
0,"What are the first names, office locations of ...","CREATE TABLE course (\n crs_code text,\n ...","SELECT T2.emp_fname, T4.prof_office, T3.crs_de..."
1,Please show the songs that have result 'nomina...,"CREATE TABLE artist (\n artist_id number,\n...",SELECT T2.song FROM music_festival AS T1 JOIN ...
2,Which teams had more than 3 eliminations?,CREATE TABLE elimination (\n elimination_id...,SELECT team FROM elimination GROUP BY team HAV...
3,"Show the names of people, and dates and venues...","CREATE TABLE people (\n people_id number,\n...","SELECT T3.name, T2.date, T2.venue FROM debate_..."
4,Tell me the the date when the first claim was ...,CREATE TABLE settlements (\n settlement_id ...,SELECT date_claim_made FROM claims ORDER BY da...


In [342]:
# Split the SQL query into lines
def table_column_of_create_table(query):
    lines = query.splitlines()

    # Initialize a list to store column names
    columns = []
    table_names = []

    # Look for "CREATE TABLE" and start capturing columns
    capture = False

    for line in lines:
        if "CREATE TABLE" in line:
            capture = True
            table_names.append(line.split()[-2])
        elif line.strip().endswith(')') or line.strip().endswith(');'):
            capture = False
        elif capture:
            column_name = line.strip().split()[0]
            if column_name in ["CONSTRAINT", "PRIMARY"]: continue
            columns.append(column_name)

    # print("Table Name:", table_names)
    # print("Columns:", columns)
    return table_names, columns


In [157]:
spider_path = 'src/spider/database'
map_table_db = {}

for folder in os.listdir(spider_path):
    schema_path = os.path.join(spider_path, folder, 'schema.sql')
    if os.path.exists(schema_path):
        with open(schema_path, 'r') as sql_file:
            sql_script = sql_file.read()
            table_names = table_column_of_create_table(sql_script)[0]
            for table in table_names: map_table_db[table] = folder

with open("src/spider/table_database_map.json", "w") as f:
    json.dump(map_table_db, f, indent=4)

In [356]:
sql_extract_token_type = {
            sqlparse.sql.IdentifierList, sqlparse.sql.Where,
            sqlparse.sql.Having, sqlparse.sql.Comparison, sqlparse.sql.Function,
            sqlparse.sql.Parenthesis, sqlparse.sql.Operation, sqlparse.sql.Case
        }

def columns_from_query(sql_query):
    # identifiers contain table name and column name
    if type(sql_query) == str:
        sql_query = sqlparse.parse(sql_query)[0]
    identifiers = []
    for token in sql_query:
        if isinstance(token, sqlparse.sql.Identifier):
            identifiers.append(token.get_real_name())
        elif hasattr(token, "tokens"):
            identifiers.extend(columns_from_query(token.tokens))
    return identifiers

def columns_by_split(sql_query:str, all_columns:list):
    columns = []
    for token in sql_query.split():
        if token[-1] == ",": token = token[:-1]
        if token in all_columns:
            columns.append(token)
    return columns

In [362]:
db_to_table_map = {}
for table, db in map_table_db.items():
    if db in db_to_table_map:
        db_to_table_map[db].append(table)
    else:
        db_to_table_map[db] = [table]


In [387]:
exists_table = [i['table'] for i in dbs]
for i,row in df.iterrows():
    table_of_query = row['Table']
    tables, all_columns = table_column_of_create_table(table_of_query)
    if tables[0] in exists_table:
        try:
            # db_path = os.path.join(spider_path, map_table_db[tables[0]])
            db = map_table_db[tables[0]]
            table_in_db = db_to_table_map[db]
            print(row['Question'])
            print(row['SQL'])
            columns = Parser(row['SQL']).columns
            expect_cols = []
            expect_table = []
            for col in columns:
                # that has join function
                if "." in col:
                    table_name, column_name = col.split('.') 
                    expect_cols.append(column_name)
                    expect_table.append(table_name)
                elif col in all_columns:
                    expect_cols.append(col)
            
            expect_cols.extend([c for c in columns_from_query(row['SQL']) if c in all_columns])
            expect_cols.extend(columns_by_split(row['SQL'], all_columns))
            expect_table.extend(Parser(row['SQL']).tables)

            expect_table = list(set(expect_table))
            expect_cols = list(set(expect_cols))
            expect_cols = [c for c in expect_cols if c not in expect_table]

            print("EXPECT COLUMNS:",expect_cols)
            print("EXPECT TABLE:", expect_table)
            
            result = filter_tables_by_description(row['Question'], column_threshold = 0.1, filter_tables = False)
            result = column_from_question(row['Question'], used_table_col=result)
            
            result_tables = []
            result_columns = []
            for t in result:
                if t in table_in_db: 
                    result_tables.append(t)
                    result_columns.extend(list(result[t].keys()))
            print("PREDICT TABLE:", result_tables)
            print("PREDICT COLUMNS:", result_columns)

            table_TP = len(set(expect_table) & set(result_tables))
            table_FP = len(set(result_tables) - set(expect_table))
            table_FN = len(set(expect_table) - set(result_tables))
            col_TP = len(set(expect_cols) & set(result_columns))
            col_FP = len(set(result_columns) - set(expect_cols))
            col_FN = len(set(expect_cols) - set(result_columns))

            
            
            try:
                table_recall = table_TP / (table_TP + table_FN)
                table_precision = table_TP / (table_TP + table_FP)
                
                col_recall = col_TP / (col_TP + col_FN)
                col_precision = col_TP / (col_TP + col_FP)

                table_f1 = 2 * (table_precision * table_recall) / (table_precision + table_recall)
                col_f1 = 2 * (col_precision * col_recall) / (col_precision + col_recall)
            except ZeroDivisionError:
                table_f1, col_f1 = "ZeroDivisionError", "ZeroDivisionError"
            print("TABLE RECALL:", round(table_recall,2), "\tCOLUMNS RECALL:", round(table_recall,2))
            print("TABLE PRECISION:", round(table_precision,2), "\tCOLUMNS PRECISION:", round(col_precision,2))
            print("TABLE F1 SCORE:", table_f1, "\tCOLUMNS F1 SCORE:", col_f1)
            print()
        except KeyError: pass

Show the names of people, and dates and venues of debates they are on the negative side, ordered in ascending alphabetical order of name.
SELECT T3.name, T2.date, T2.venue FROM debate_people AS T1 JOIN debate AS T2 ON T1.debate_id = T2.debate_id JOIN people AS T3 ON T1.negative = T3.people_id ORDER BY T3.name
EXPECT COLUMNS: ['negative', 'name', 'date', 'people_id', 'venue', 'debate_id']
EXPECT TABLE: ['people', 'debate_people', 'debate']
PREDICT TABLE: ['people']
PREDICT COLUMNS: ['Name', 'Nationality', 'name']
TABLE RECALL: 0.33 	COLUMNS RECALL: 0.33
TABLE PRECISION: 1.0 	COLUMNS PRECISION: 0.33
TABLE F1 SCORE: 0.5 	COLUMNS F1 SCORE: 0.2222222222222222

Find the number of members living in each address.
SELECT COUNT(*), address FROM member GROUP BY address
EXPECT COLUMNS: ['address']
EXPECT TABLE: ['member']
PREDICT TABLE: ['shop', 'member']
PREDICT COLUMNS: ['Address', 'Num_of_staff', 'Member_ID', 'Name', 'Membership_card', 'Age', 'Level_of_membership', 'Address']
TABLE RECALL: 1.0 

In [379]:
i

6993

In [381]:
# Define the expected and predicted values as lists
expected = ["a", "b"]
predicted = ["b", "c", "d", "e"]

# Count True Positives (TP), False Positives (FP), and False Negatives (FN)
TP = len(set(expected) & set(predicted))
FP = len(set(predicted) - set(expected))
FN = len(set(expected) - set(predicted))

# Calculate Recall and Precision
recall = TP / (TP + FN)
precision = TP / (TP + FP)

# Print the results
print("Recall:", recall)
print("Precision:", precision)


Recall: 0.5
Precision: 0.25
